# Build the Neural Network

In [6]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Get Device for Training

In [7]:
# if available, we want to be able to train our model on a hardware accelerator like the GPU or MPS.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


# Define the Class

In [8]:
# define neural network by subclassing nn.Module
# Base class for all neural network modules
class NeuralNetwork(nn.Module):
    def __init__(self):
        # Initialize internal Module state, shared by both nn.Module and ScriptModule.
        super().__init__()
        
        # This is Model Layers
        # convert each 2D 28x28 image into a contiguous array of 784 pixel values
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            # applies a linear transformation on the input using its stored weights and biases.
            nn.Linear(28*28, 512),
            # Non-linear activations
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    # Every nn.Module subclass implements the operations on input data in the forward method.
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [9]:
# create an instance of NeuralNetwork, and move it to the device, and print its structure.
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [10]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2], device='mps:0')


# Model Layers

In [11]:
input_image = torch.rand(3,28,28)
print(input_image.size())


# nn.Flatten
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())


# nn.Linear
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())


# nn.ReLU
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")


nn.Sequential
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)


# nn.Softmax
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

torch.Size([3, 28, 28])
torch.Size([3, 784])
torch.Size([3, 20])
Before ReLU: tensor([[ 0.2215, -0.2454, -0.6104, -0.6091, -0.4696,  0.1986, -0.2132, -0.1406,
         -0.4442,  0.3264,  0.0574,  0.1953,  0.3970, -0.1245,  0.0137, -0.4407,
         -0.1408,  0.3928, -0.0332,  0.1689],
        [ 0.5270, -0.0647, -0.4156, -0.3075, -0.5541, -0.0771,  0.2963, -0.1519,
         -0.0755,  0.3072, -0.3078,  0.1890,  0.0574, -0.0091, -0.1008, -0.1593,
         -0.1094,  0.4573, -0.2499,  0.3267],
        [ 0.1373,  0.1955,  0.0792, -0.2364, -0.4897,  0.1398,  0.2699, -0.0929,
         -0.1908,  0.2720, -0.2013,  0.2885,  0.5414, -0.0558, -0.5844,  0.0958,
          0.2053,  0.3202,  0.0922,  0.4062]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.2215, 0.0000, 0.0000, 0.0000, 0.0000, 0.1986, 0.0000, 0.0000, 0.0000,
         0.3264, 0.0574, 0.1953, 0.3970, 0.0000, 0.0137, 0.0000, 0.0000, 0.3928,
         0.0000, 0.1689],
        [0.5270, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2963, 0.00

# Model Parameters

In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0137, -0.0312, -0.0052,  ..., -0.0096, -0.0019, -0.0123],
        [-0.0301, -0.0251, -0.0212,  ...,  0.0015,  0.0336,  0.0244]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0323, -0.0166], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0142, -0.0215,  0.0148,  ..., -0.0084, -0.0408, -0.0015],
        [ 0.0023,  0.0157,  0.0129,  ...,  0.0236,  0.0340,  0.0190]],
       device='mps:0', grad_fn=<Slice